<a href="https://colab.research.google.com/github/fyqxtz/2025-HAI-summer-assignment-2/blob/main/HAI_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [ ]:
# download the MINST data

batch_size = 64

transforms_train = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) # 픽셀 정규화
])

train_dataset = datasets.MNIST(root="./dataset", train=True, download=True, transform=transforms_train)

dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=4)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
latent_dim = 100 # 100차원 잠재공간벡터

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # DENSE + RELU로 한 블럭을 구성함
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, int(np.prod(img_shape))), #최종 출력 크기대로 변환후
            nn.Tanh() # -1 ~ 1 사이 값으로 변경 (정규화한대로)
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(), #Discriminator은 0~1범위의 판별결과를 반환하니까 시그모이드를 이용
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [ ]:
#하이퍼파라미터들
channels = 1
img_size = 28

img_shape = (channels, img_size, img_size)

# learning rate
lr = 0.0002

# 모멘텀
b1 = 0.5
b2 = 0.999

generator = Generator()
discriminator = Discriminator()

# 오차함수
adversarial_loss = nn.BCELoss()

# optimizer
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

cuda = True if torch.cuda.is_available() else False

if cuda :
  generator.cuda()
  discriminator.cuda()
  adversarial_loss.cuda()

In [ ]:
import time

# 에포크수
n_epochs = 200


sample_interval = 5 # 5에포크마다 샘플 저장
start_time = time.time()
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        #실제 -> 1, 가짜 -> 0으로 라벨링
        real = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        real_imgs = Variable(imgs.type(Tensor))

        ###생산자 학습

        optimizer_G.zero_grad()

        #노이즈 생성(100차원)
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        #이미지 생성
        generated_imgs = generator(z)

        #loss값계산.
        g_loss = adversarial_loss(discriminator(generated_imgs), real)

        #업데이트
        g_loss.backward()
        optimizer_G.step()

        ###판별자 학습

        optimizer_D.zero_grad()

        #앞서 설정해둔 real, fake 이용
        real_loss = adversarial_loss(discriminator(real_imgs), real)
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        #업데이트
        d_loss.backward()
        optimizer_D.step()

    if (epoch+1) % sample_interval == 0:
        # 생성된 이미지 중에서 25개만 선택하여 5 X 5 격자 이미지에 출력
        save_image(generated_imgs.data[:25], f"data{epoch}.png", nrow=5, normalize=True)

    # 하나의 epoch이 끝날 때마다 로그(log) 출력
    print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[Epoch 0/200] [D loss: 0.321019] [G loss: 2.363057] [Elapsed time: 50.63s]
[Epoch 1/200] [D loss: 0.499459] [G loss: 2.953451] [Elapsed time: 98.88s]
[Epoch 2/200] [D loss: 0.266184] [G loss: 1.455999] [Elapsed time: 148.44s]
[Epoch 3/200] [D loss: 0.472569] [G loss: 0.992509] [Elapsed time: 196.85s]
[Epoch 4/200] [D loss: 0.781897] [G loss: 2.520082] [Elapsed time: 246.42s]
[Epoch 5/200] [D loss: 0.535278] [G loss: 1.795845] [Elapsed time: 295.97s]
[Epoch 6/200] [D loss: 0.576857] [G loss: 1.002638] [Elapsed time: 344.13s]
[Epoch 7/200] [D loss: 0.583426] [G loss: 0.940636] [Elapsed time: 393.77s]
[Epoch 8/200] [D loss: 0.548840] [G loss: 1.305569] [Elapsed time: 442.32s]
[Epoch 9/200] [D loss: 0.603130] [G loss: 0.711787] [Elapsed time: 492.07s]
[Epoch 10/200] [D loss: 0.457685] [G loss: 1.288728] [Elapsed time: 540.94s]
[Epoch 11/200] [D loss: 0.598734] [G loss: 0.768444] [Elapsed time: 589.96s]
[Epoch 12/200] [D loss: 0.593941] [G loss: 1.313183] [Elapsed time: 639.57s]
[Epoch 13/2


<img src = "https://drive.google.com/uc?id=1H77alZcSY-1fee3TGhK9cNWN7Rzrm8eK" height = 100 width = 100>
<img src = "https://drive.google.com/uc?id=1l6ON0WHDTNXQvRWgnjXYtQrqafmWr6fK" height = 100 width = 100>
<img src = "https://drive.google.com/uc?id=1f9FPSfO8oTuR9OESyWM4xDmkCmZRh0US" height = 100 width = 100>
<img src = "https://drive.google.com/uc?id=1dPt0xnlHW37TjwFts16nEhh21PLQqX5T" height = 100 width = 100>
<img src = "https://drive.google.com/uc?id=1xdQqbk-VvD3opW4IPDK8Lcwb9BJN_Hv0" height = 100 width = 100>

epoch
5, 50, 100, 150, 200